In [3]:
# import textstat
import pandas as pd
import spacy

ModuleNotFoundError: No module named 'spacy'

In [2]:
# Load English SpaCy model
nlp = spacy.load("en_core_web_sm") #In terminal: python -m spacy download en_core_web_sm

# Download concreteness lexicon
# Direct URL to the CSV file (latest version, Conc.M = concreteness mean)
url = "hf://datasets/StephanAkkerman/MRC-psycholinguistic-database/mrc_psycholinguistic_database.csv"
# Load into pandas
lexicon = pd.read_csv(url)
# Clean and keep only useful columns
lexicon = lexicon.rename(columns={"Word": "word", "Conc.M": "concreteness"})
# lexicon = lexicon[["word", "concreteness"]]
# lexicon["word"] = lexicon["word"].str.lower()
lexicon.columns

NameError: name 'spacy' is not defined

In [2]:
pip install -r "/home/onyxia/work/Phoetry-1/requirements.txt"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.5/151.5 KB 2.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 KB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 KB 19.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 30.2 MB/s eta 0:00:00a 0:00:01
Using legacy 'setup.py install' for argparse, since package 'wheel' is not installed.
  Running setup.py install for argparse ... error
  error: subprocess-exited-with-error
  
  × Running setup.py install for argparse did not run successfully.
  │ exit code: 1
  ╰─> [11 lines of output]
      running install
      /home/onyxia/work/Phoetry-1/env_ph/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
        warnings.warn(
      running build
      running build_py
      creating b

In [ ]:
# Define the function

def analyze_text(text):
    doc = nlp(text)
    words = [token.text.lower() for token in doc if token.is_alpha]

    # Readability (Flesch Reading Ease)
    readability = textstat.flesch_reading_ease(text)

    # Imageability & Concreteness (averages)
    imageability_scores = [
        lexicon.loc[lexicon['word'] == word, 'imageability'].values[0]
        for word in words if word in lexicon['word'].values and 'imageability' in lexicon.columns
    ]
    concreteness_scores = [
        concrete.loc[lexicon['word'] == word, 'concreteness'].values[0]
        for word in words if word in concrete['word'].values and 'concreteness' in concrete.columns
    ]

    avg_imageability = sum(imageability_scores) / len(imageability_scores) if imageability_scores else None
    avg_concreteness = sum(concreteness_scores) / len(concreteness_scores) if concreteness_scores else None

    # Syntactic simplicity (average number of words per sentence)
    avg_sentence_length = sum(len(sent) for sent in doc.sents) / len(list(doc.sents))

    return {
        "Readability (Flesch)": round(readability, 2),
        "Imageability": round(avg_imageability, 2) if avg_imageability else "N/A",
        "Concreteness": round(avg_concreteness, 2) if avg_concreteness else "N/A",
        "Syntactic simplicity (avg sentence length)": round(avg_sentence_length, 2)
    }


In [ ]:
# Exemple d'utilisation
poems=pd.read_excel("C:/Users/Annek/Documents/MSDS/semestre 2/ml for nlp/Projet/evaluation/poem_classic.xlsx")

poems["scores"]=poems["explicit prompt"].apply(lambda x: analyze_text(x))

In [ ]:
poems["scores"]